# Introduction

A notebook that exploit class definition in 'source' folder to deal with Training and Testing Analysis of ORCT (_Optimal Randomized Classification Trees_ Blanquero et Al.2018).

### Remark
* This class after preprocessing phase can manage any kind of dataset

In [ ]:
# dataframe management
import pandas as pd
import math
import numpy as np
import sklearn
import json
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from functools import reduce # Valid in Python 2.6+, required in Python 3
import operator
from pyomo.environ import *
from pyomo.opt import SolverFactory
from source.util import *
from source.maths import *
from source.orct import *
infinity = float('inf')

# Preprocessing of dataset
Let's load the Iris dataset:

In [2]:
iris = pd.read_csv('... .csv') #IrisCategorical.csv
iris = iris.drop('Id', axis=1)
iris_std = iris.copy()
iris.head(5)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


Commands to manage generic dataset with labelled column 'Classes'

In [3]:
iris.rename(columns={'Species':'Classes'}, inplace=True)
iris_std.rename(columns={'Species':'Classes'},inplace=True)

In [4]:
scaler = MinMaxScaler() # also MaxAbsScaler()

In [5]:
#Preprocessing: we get the columns names of features which have to be standardized
columns_names = list(iris)
index_features = list(range(0, 4))
index_instances = list(range(0,150))

#The name of the classes K
classes = iris_std['Classes'].unique().tolist()
classes_en = [i for i in range(len(classes))] 

#Encoder processing
le = preprocessing.LabelEncoder()
le.fit(iris_std['Classes'])

iris_std['Classes'] = le.transform(iris_std['Classes']) 

#Scaling phase
iris_std[columns_names[0:4]] = scaler.fit_transform(iris_std[columns_names[0:4]])

iris_std.head(1)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Classes
0,0.222222,0.625,0.067797,0.041667,0


Splitting the dataset between __Training & Testing Sets__

In [6]:
df = iris_std[columns_names[:-1]]
y = iris_std[columns_names[4]]
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.25)
df_train = pd.concat([X_train, y_train], axis=1, join_axes=[X_train.index])

In [7]:
I_in_k = inst_class(df_train)
def I_k(model,i):
    if i==0:
        return I_in_k[0]
    elif i==1:
        return I_in_k[1]
    elif i==2:
        return I_in_k[2]

In [20]:
init_a = np.random.uniform(low=-1.0, high=1.0, size=None)
init_mu = np.random.uniform(low=-1.0, high=1.0, size=None)
init_C = np.random.uniform(low=0.0, high=1.0, size=None)
init_P = np.random.uniform(low=0.0, high=1.0, size=None)
init_p = np.random.uniform(low=0.0, high=1.0, size=None)
p = ORCT_2(df_train,I_in_k,I_k,[init_a,init_mu,init_C,init_P,init_p])

In [21]:
# To solve our model
p.solve()

In [22]:
# Value of the Objective function
p.value_obj()

0.17893963967967513

In [23]:
p.extraction_va()
ciccio = p.predicted_lab(X_test)